# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import PIL.Image as Image
from collections import OrderedDict
from copy import deepcopy
from time import time
from workspace_utils import keep_awake, active_session
# Importing pandas as data management seems easier when in dataframe
import pandas as pd
import seaborn as sns
import json
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [3]:
# TODO: Define your transforms for the training, validation, and testing sets
# While training model, RandomRotation, RandomResized etc. has been used to train model on randomly organised images.
# This will avoid overfitting or reduce the chances of model remembering the images

data_transforms = {
   'train': transforms.Compose([
       transforms.RandomRotation(30),
       transforms.RandomResizedCrop(224),
       transforms.RandomHorizontalFlip(),
       transforms.ToTensor(),
       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
   ]),
   'valid': transforms.Compose([
       transforms.Resize(256),
       transforms.CenterCrop(224),
       transforms.ToTensor(),
       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
   ]),
   'test': transforms.Compose([
       transforms.Resize(256),
       transforms.CenterCrop(224),
       transforms.ToTensor(),
       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
   ])
}

# TODO: Load the datasets with ImageFolder

image_datasets = {
   'train': datasets.ImageFolder(train_dir, transform=data_transforms['train']),
   'valid': datasets.ImageFolder(valid_dir, transform=data_transforms['valid']),
   'test': datasets.ImageFolder(test_dir, transform=data_transforms['test'])
}


# TODO: Using the image datasets and the trainforms, define the dataloaders

batch_size = 32
dataloaders = {
   'train': data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True, num_workers=1),
   'valid': data.DataLoader(image_datasets['valid'], batch_size=batch_size, shuffle=True, num_workers=1),
   'test': data.DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=True, num_workers=1)
}

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:


with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

print("Total number of output: %d"%len(cat_to_name))

Total number of output: 102


In [5]:
num_classes = len(cat_to_name.keys())
#print('Number of classes to learn: {}'.format(num_classes))

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [6]:
# TODO: Build and train your network
# Creating a model from pretrained vgg16 form models library

model = models.vgg16(pretrained=True)

# freeze the pretrained model params
for param in model.features.parameters():
    param.requires_grad = False

# obtain & define various params for the classifier, build classifier
# & update pre-trained model to use our newly-minted classifier

network_input_size = 224 * 224

# some models go straight to layers, no indices

try:
    classifier_input_size = model.classifier[0].in_features
except:
    classifier_input_size = model.classifier.in_features
    

classifier_output_size = num_classes
classifier_hidden_layer_sizes = [4096, 2048, 1024]
learning_rate = 0.001
dropout_rate = 0.25
epochs = 10

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:04<00:00, 116780793.87it/s]


In [7]:
# Function to build the network as per our requirement

def build_network(in_size, hidden_layers, out_size):
    constructed_layers = OrderedDict()
    layers_input_output_joined = zip([in_size] + hidden_layers, hidden_layers)

    for i, (layer_in, layer_out) in enumerate(layers_input_output_joined):
        constructed_layers['{}-fc'.format(i)] = nn.Linear(layer_in, layer_out, bias=True) # introduce bias term for perf. boost
        constructed_layers['{}-relu'.format(i)] = nn.ReLU()
        constructed_layers['{}-dropout'.format(i)] = nn.Dropout(p=dropout_rate)

   # input of the last layer is the last hidden layer
   # introduce bias again conforming to 'best practices'
    constructed_layers['output'] = nn.Linear(hidden_layers[-1], classifier_output_size, bias=True)

   # smooth out the last layer
    constructed_layers['output-smoothed'] = nn.LogSoftmax(dim=1)
    classifier = nn.Sequential(constructed_layers)
    return classifier

In [8]:
#print(classifier_input_size, classifier_hidden_layer_sizes, classifier_output_size)

In [9]:
model.classifier = build_network(classifier_input_size, 
                                 classifier_hidden_layer_sizes, 
                                 classifier_output_size)
#model.classifier

## Training and Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [10]:
# Function to train the model
def train(model, loader, criteria, optimizer, device):
    model.train()
    train_loss = 0.0
    train_correct = 0
    for x_train, y_train in loader:
        x_train, y_train = x_train.to(device), y_train.to(device)
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            output = model.forward(x_train)
            loss = criteria(output, y_train)
            loss.backward()
            optimizer.step()
            _,predictions = torch.max(output, 1)
        train_loss += loss.item()
        train_correct += (predictions == y_train.data).type(torch.FloatTensor).mean()
    epoch_loss = train_loss / len(loader)
    epoch_accuracy = train_correct / len(loader)
    return epoch_loss, epoch_accuracy

In [11]:
# TODO: Do validation on the test set
def validation(model, loader, criteria, optimizer, device):
    model.eval()
    valid_loss = 0.0
    valid_correct = 0

    for x_valid, y_valid in loader:
        x_valid, y_valid = x_valid.to(device), y_valid.to(device)

        with torch.set_grad_enabled(False):
            output = model.forward(x_valid)
            loss = criteria(output, y_valid)
            valid_loss += loss.item()
            _,predictions = torch.max(output, 1)
        valid_correct += (predictions == y_valid.data).type(torch.FloatTensor).mean()

    epoch_loss = valid_loss / len(loader)
    epoch_accuracy = valid_correct / len(loader)    
    return epoch_loss, epoch_accuracy

In [12]:
def testing(model, loader, criteria, optimizer, device):
    model.eval()
    test_loss = 0.0
    test_correct = 0

    for x_test, y_test in loader:
        x_test, y_test = x_test.to(device), y_test.to(device)

        with torch.set_grad_enabled(False):
            output = model.forward(x_test)
            loss = criteria(output, y_test)
            test_loss += loss.item()
            _,predictions = torch.max(output, 1)
        test_correct += (predictions == y_test.data).type(torch.FloatTensor).mean()

    test_loss = test_loss / len(loader)
    test_accuracy = test_correct / len(loader)    
    return test_loss, test_accuracy

In [13]:
# do training and validation
def perform_training_and_validation(model, epochs, device, criteria, optimizer):
    best_model_weights_so_far = deepcopy(model.state_dict())
    best_model_accuracy_so_far = 0.0

    model.to(device)
    for epoch in range(epochs):
        print('Current Epoch: {}/{}'.format(epoch+1, epochs))

        epoch_train_loss, epoch_train_acc = train(model, dataloaders['train'], criteria, optimizer, device)
        print("Train Loss : {:.4f} Train Accuracy: {:.4f}".format(epoch_train_loss,epoch_train_acc))

        epoch_val_loss, epoch_val_acc = validation(model, dataloaders['valid'], criteria, optimizer, device)
        print("Validation Loss : {:.4f} Validation Accuracy {:.4f}".format(epoch_val_loss,epoch_val_acc))
        print('-' * 10)
        if epoch_val_acc > best_model_accuracy_so_far:
            best_model_accuracy_so_far = epoch_val_acc

    best_model_weights_so_far = deepcopy(model.state_dict())
    print('Training Complete')
    print('Best Model Accuracy: {}'.format(best_model_accuracy_so_far))
    return best_model_accuracy_so_far, best_model_weights_so_far

In [ ]:
# run the training and validation steps above

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
criteria = nn.NLLLoss() # according to pytorch docu, useful to train classification with C classes
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)
# optimizer = optim.SGD(model.classifier.parameters(), lr = 0.005, momentum = 0.5)
with active_session():
    best_model_acc, best_model_weights = perform_training_and_validation(model, 2, device, criteria, optimizer)
    model.load_state_dict(best_model_weights)

Current Epoch: 1/2
Train Loss : 3.1798 Train Accuracy: 0.2573
Validation Loss : 1.5762 Validation Accuracy 0.5447
----------
Current Epoch: 2/2


In [ ]:
# testing
with active_session():
    test_loss_n_accuracy = testing(model, dataloaders['test'], criteria, optimizer, device)
    
print("Test Loss: %.2f | Test Accuracy: %.2f "%test_loss_n_accuracy) 

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
#Add classes to model
model.class_to_idx = image_datasets['train'].class_to_idx


In [ ]:
# save model and other information
checkpoint={'classifier': model.classifier,
            'class_to_idx' : model.class_to_idx,
            'epochs': epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}
torch.save(checkpoint,'checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_n_rebuild(checkpointFilePath):
    model_state = torch.load(checkpointFilePath)
    model = models.vgg16(pretrained=True)
    model.classifier = model_state['classifier']
    model.load_state_dict(model_state['model_state_dict'])
    model.class_to_idx = model_state['class_to_idx']
    
    
    return model

In [ ]:
new_model = load_n_rebuild('checkpoint.pth')
new_model = new_model.to(device)
#print(new_model)

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:

def process_image(image_location):
    
    image = Image.open(image_location)
    
    # Resize and crop out the center 224x224 square
    aspect = image.size[0]/image.size[1]
    if aspect > 0:
        image.thumbnail((10000, 256))
    else:
        image.thumbnail((256, 10000))
        
    left_margin = (image.width-224)/2
    top_margin = (image.height-224)/2
    
    image = image.crop((left_margin, top_margin, left_margin+224, top_margin+224))

    # Now normalize...
    image = np.array(image)/255
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = (image - mean)/std

    # Move color channels to first dimension as expected by PyTorch
    image = image.transpose((2, 0, 1))
   

    return image
    
    
    


To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    

    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    #image = Image.open(image_path)
    image = process_image(image_path)
    print("Image shape after processing: ",image.shape)
    image = torch.from_numpy(image).type(torch.FloatTensor)
    image = image.unsqueeze(0)  
    model = model.cpu()
    logps = model.forward(image)
    ps = torch.exp(logps)
    top_p, top_class = ps.topk(topk)
        
    return top_p, top_class

In [ ]:
image_path = 'flowers/test/19/image_06159.jpg'

img = process_image(image_path)

In [ ]:

probs, classes = predict(image_path, new_model)



In [ ]:

# Function returns cuda tensors, convert to numpy array
probs = probs.type(torch.FloatTensor).to('cpu').detach().numpy()
classes = classes.type(torch.FloatTensor).to('cpu').detach().numpy()
classes = classes.astype(int)
classes = classes.astype(str)

In [ ]:
#print(classes)
#print(probs)

In [ ]:
classes = [item for sublist in classes for item in sublist]
probs = [item for sublist in probs for item in sublist]
#print(classes)
#print(probs)

In [ ]:
class_names=[]
class_id=[]
class_prob=[]
for i in range(len(classes)):
    if classes[i]!='0':
        class_id.append(classes[i])
        class_names.append(cat_to_name[classes[i]])
        class_prob.append(probs[i])
    

In [ ]:
classesTup = list(zip(class_id,class_names,class_prob))
#classesTup

In [ ]:
df = pd.DataFrame(classesTup, columns = ['Class_ID','Class_Name','Class_Prob']) 
df = df.sort_values(by=['Class_Prob','Class_Name'],ascending = False)
likely_class = df[df['Class_Prob']==df['Class_Prob'].max()]['Class_Name']
predicted_class=likely_class[0]

In [ ]:

ax = sns.barplot(x='Class_Prob', y='Class_Name', data=df)
ax.set_xlabel('Class_Prob')
ax.set_title(predicted_class)
ax.set_xlabel('Probabilities')
ax.set_ylabel('Classes')
imshow(img, ax=None, title=predicted_class)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.